# Pytorch 数据预处理与数据集加载
October 7, 2021
***
在Pytorch构建神经网络对数据集进行训练和测试前，需要对数据进行预处理操作。预处理的主要任务就是将图片、文字、视频等形式的训练/测试集转换为Python能够读取的数据。具体地，需要将这些数据转换为张量(tensor)  
在Pytorch框架中，对于数据的预处理具有严格的程序规范。包括**创建对象**,**说明方法**等。 

这部分内容主要举一些例子，并结合代码，详细说明Pytorch的数据预处理.

# 0.概述
Pytorch的数据预处理可以概括为“三步走”  
+ 创建Dataset类 - 将**普通的数据**变成**Pytorch认识的数据集**  
+ 创建Dataloader类 - 对数据进行“斗地主中”**洗牌**,**发牌**等操作（对应Shuffle,Batch）
+ 转换Transform

# 1.Dataset类与数据集读取
Dataset类是将**普通的数据**转换为**Pytorch认识的数据**的一个类
## 1.1 数据集及其格式
一般情况下，一个完整的数据集应该由以下两部分组成
+ 数据 （如文字，图像，视频序列）
+ 标签 （如类别，像素位置等）  
数据集中的数据形式，随着不同数据集的变化各不相同,在使用时需要根据指定的格式进行预处理  
下面举几个例子，说明一般数据集具有的数据结构特点： 

### CIFAR10数据集  
CIFAR10数据集包含60000张32×32大小的图像，这些图像来自10个常见物品（如飞机、汽车、鸟、狗等）
<img src="https://gimg2.baidu.com/image_search/src=http%3A%2F%2Fyqfile.alicdn.com%2F8fe03194fc0933b8ed08fcfbfcca2bdc44201da7.png&refer=http%3A%2F%2Fyqfile.alicdn.com&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=jpeg?sec=1636166953&t=52a168c83244bf9191d820dfafdeddb2">

+ 数据：像素数据(0 ~ 255,三通道)使用numpy数组存储。32×32大小，10000张图，共形成10000×3072的numpy数组，存整形变量
+ 标签：标签数据(0 ~ 9),形成列表list。分别表示训练数据所对应的类别


### MNIST手写数字识别数据集
MNIST手写数字识别数据集包含60000张训练集以及10000张测试集，这些图像来自10个手写数字
<img src="https://img2.baidu.com/it/u=2939531682,1020933556&fm=26&fmt=auto">
MNIST数据集官网并不直接提供手写数字的图像文件(.img等格式)。而是将他们在内存中的地址关系以及地址单元中的内容进行存储，需要写程序将MNIST读取。存取规则如下：  
+ 数据：像素数据(0 ~ 255)使用一个字节地址存储，体现在图像上为**黑白图片**  
+ 标签：标签数据(0 ~ 9)使用一个字节地址存储


### 中文文本分类数据集
<img src="./截屏2021-10-07 15.23.55.png">  
+ 数据：如图所示，数据为文字数据，每一行为一个数据，行内使用 " _ ! _ " 区分  
+ 标签：分别表示新闻ID,分类Code，分类名称，新闻字符串，新闻关键词


### 人体姿态识别数据集
<img src="http://sam.johnson.io/research/images/dataset_et/im00004.jpg"> <img src="http://sam.johnson.io/research/images/dataset_et/viz/im00004.jpg" width=20% height=20%>
+ 数据：以图片(im000x.jpg)格式给出，共2000张图，每张
+ 标签：以表格(joints.mat)格式给出，大小为3×14×2000，存每个图像14个关键点的坐标位置

## 1.2 自建数据集
然而，在大多数场景下，无法使用Pytorch可以直接支持的数据集进行操作。因此需要根据一定格式，创建易于读取的数据集。  
在创建数据集的过程中，所有数据需要保证在一个文件夹中。  
注意：在任何系统中，`~/`符号表示系统主目录，而`./`表示当前文件所在的根目录，用于表示**相对路径**  
以人脸姿态数据集为例，假设数据存放的相对路径`./faces`为保证程序正常运行，该目录下应包含：
+ 数据：以图片形式给出：`ABC.jpg`等
+ 标签：以`data.csv`形式给出，形式如下:  

| 图片文件名    | 标签 |
| ----------- | ----------- |
| ABC.jpg     | 1       |
| BCD.jpg     | 2        |

具体地，由于数据中，每个点包含的标签信息不再是**类别信息**，而是14个关键点的(x,y)坐标，故`data.csv`中具体格式应当为  

| **图片文件名**             | **part0_x** | **part0_y** | **part1_x** | **part1_y** | **.......** | **part67_y** |
|-----------------------|-------------|-------------|-------------|-------------|-------------|--------------|
| `0805personalli01\.jpg` | 27          | 83          | 27          | 98          |             |              |
| `1084\.\.\.\.jpg `      | 70          | 236         | 71          | 257         |             |              |
| \.\.\.\.              |             |             |             |             |             |              |

接着，开始读取数据集中的数据和标签。在代码中输入以下内容

In [46]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt
import pandas as pd

 在成功引入以下包后，在上一个文本框中的 `In [ ]:` 可以正常显示数字  
 接着，输入如下代码，读取表格中的文件

In [10]:
labels = pd.read_csv('./faces/face_landmarks.csv')

读取成功后，查看label的类型,输入`type(labels)`，应显示`pandas.core.frame.DataFrame`

In [11]:
type(labels)

pandas.core.frame.DataFrame

接着，对`.csv`格式文件进行读取，以第1行数据为例

In [16]:
n = 0
image_name = labels.iloc[n,0]
image_mark = labels.iloc[n,1:]
image_mark = np.asarray(image_mark)
image_mark = image_mark.astype('int').reshape(-1,2)

#print(image_name)
#print(image_mark)

读取成功后，若可以查看输出的表格信息，则说明`.csv`格式文件成功读取

## 1.3 Dataset类
在Pytorch中，用于生成数据集的类继承Dataset父类。以便将数据集转换成Pytorch可以识别的形式。  
其中，常见数据集可以直接表示为Dataset类，无需手动生成。这样的数据集包括
+ MNIST
+ COCO
+ Captions
+ Detection
+ LSUN
+ ImageFolder
+ Imagenet-12
+ CIFAR
+ STL10
+ SVHN
+ PhotoTour

> PyTorch domain libraries provide a number of pre-loaded datasets (such as FashionMNIST) that subclass torch.utils.data.Dataset and implement functions specific to the particular data. They can be used to prototype and benchmark your model. 
https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

将读取文件并构造数据集的方法封装成一个类。该类需要继承`Dataset`父类，又要包含以下方法
+ `__len__`  定义获取数据集的总长度，使用`len(object_name)`获取
+ `__getitem__` 定义获取数据的方法，即使用`[]`索引访问  

### 下面以`某人脸数据集`为例，说明Pytorch不认识数据集的构建方法
参考代码如下

In [13]:
class FaceLabelDataset(Dataset):
    def __init__(self,csv,img_path):
        """
        Args:
        csv(string):Path to the csv file with labels
        img_path(string):Path to the image folder
        """
        self.labels = pd.read_csv(csv)
        self.img_path = img_path
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        a = self.labels.iloc[idx,1:]
        a = np.array([a])
        a = a.reshape(-1,2)
        return a

将`FaceLabelDataset`例化，生成实体

In [14]:
fd = FaceLabelDataset('./faces/face_landmarks.csv','./faces/')

读取信息

In [33]:
len(fd)

69

In [15]:
fd[3][4]

array([73, 220], dtype=object)

### 下面以`MNIST`数据集为例，说明Pytorch认识数据集的构建方法
以 MNIST为例，这部分数据集直接在Pytorch中为封装好的Dataset类，直接引用代码如下  
**请注意：在运行该代码前，需要保证import部分已经运行，否则将无法使用torchvision等包**

In [24]:
train = datasets.MNIST(root='./MNIST_pytorch/',train=True,transform=True)

若提示未下载，在argument中加入 `download=True` 声明即可  
对数据集进行索引，输入如下代码

In [41]:
train.train_labels[1344]

tensor(8)

In [38]:
train.classes

['0 - zero',
 '1 - one',
 '2 - two',
 '3 - three',
 '4 - four',
 '5 - five',
 '6 - six',
 '7 - seven',
 '8 - eight',
 '9 - nine']

In [44]:
a = train.train_data[1344]
a

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  41, 111, 232, 196,
          74,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   

对数据可视化，正确结果应当弹出一张图片，显示数字`8`

In [52]:
def tensorImageShow(tensorin):
    show = ToPILImage()
    show(tensorin).show()

tensorImageShow(a)

# 2.DataLoader类与数据集基本操作
DataLoader类是接续Dataset类后，为了完成数据集进行进一步操作，如**分批 - batch_size, 洗牌 - shuffle**等

  以例化的人脸数据集`fd`为例

In [53]:
fdl = torch.utils.data.DataLoader(fd,batch_size=5,shuffle=True)

# 参考
### Pytorch 数据预处理过程
+ https://zhuanlan.zhihu.com/p/173727709
+ https://zhuanlan.zhihu.com/p/95504432
+ https://blog.csdn.net/kdongyi/article/details/103272579

### 数据集官方说明
+ https://www.cs.toronto.edu/~kriz/cifar.html CIFAR数据集
+ http://yann.lecun.com/exdb/mnist/ MNIST数据集
+ https://www.kaggle.com/sinakaraji/covid-vaccination-vs-death/version/1 covid19病例数据集
+ https://github.com/aceimnorstuvwxz/toutiao-text-classfication-dataset 今日头条新闻标题分类数据集
+ http://sam.johnson.io/research/lsp.html 人体姿态识别数据集

### 数据集处理教程参考
+ https://www.jianshu.com/p/e7c286530ab9 MNIST数据集的处理
+ https://zhuanlan.zhihu.com/p/130673468 Pytorch加载自己的数据集
+ https://blog.csdn.net/qq_32896115/article/details/90311697 MNIST与自建数据集处理方法的比较